# Introduction/Business Problem

I'm planning to move to Toronot from Montreal. I would like to move to the neighbourhood similar to where I currenlty live. For that purpose I would need to use Foursquare API to get the venues in my current neighbourhood. I would need to use clustering to determine cluster labels for Montreal.

Next, using Foursquare API, I'll fetch venues data for Toronoto. Instead of clustering I'll use classification methodology to classify Toronto's neighbourhoods given labels from Montreal neighbourhood clusterization. Then I'll choose same clusters in Toronto that I like in Montreal. And that would be my neighborhood of first choice to consider.


# Data 

1. I'll be using Foursquare API Venue

https://api.foursquare.com/v2/venues/explore
    
2. I'll do same clustering exercise as I did for clustering Toronto neighbourhood, but this time for Montreal city.

Postal Codes for Montreal:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H
    
2. I'll use K-Nearest Neighbour Classifier to predict neghbour cluster given venues in 500m proximity.   